# How to use the LabelGenerator classes with timebased labelling

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Make sure you are in the drone_steering directory

cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)
    
os.getcwd()

0 E:\Pascal\DSR\drone_steering\models\supporting_notebooks\class_documentation
1 E:\Pascal\DSR\drone_steering\models\supporting_notebooks
2 E:\Pascal\DSR\drone_steering\models
3 E:\Pascal\DSR\drone_steering


'E:\\Pascal\\DSR\\drone_steering'

In [11]:
data_dir = "data/gesture/"

In [13]:
file_name_labels = "labels_timebased/labels_flip_p_01.csv"
df = pd.read_csv(data_dir + file_name_labels)
df.head()

,real_start,real_end,diff,label
0,18.70,20.45,-0.25,3
1,20.40,22.40,0.00,3
2,22.35,24.50,0.15,3
3,26.65,28.55,-0.10,3
4,30.70,32.60,-0.10,3


In [14]:
file_name_feature = "features/features_flip_p_01_120.csv"
feature_data = pd.read_csv(data_dir + file_name_feature)
feature_data.head()

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_last_frame,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,362.916626,231.219007,342.953378,283.541312,348.772704,166.245393,372.237121,285.299184,0,0.0,270.158875,224.355883,294.862550,284.492503,277.058700,166.476502,269.385661,277.767283
1,364.180072,231.006023,344.422331,283.465604,347.642222,165.250143,372.912682,285.427752,106,106.0,270.456436,220.697018,296.207271,283.810974,277.393757,165.318363,268.442911,278.002018
2,364.463694,232.132638,344.786367,284.089036,349.140680,165.521388,372.737987,285.656638,110,216.0,269.989188,222.659280,296.168246,284.376554,276.986625,165.356502,268.550011,278.863787
3,361.394807,230.397043,339.481446,282.356428,347.711305,164.712134,373.510148,283.864464,109,325.0,269.539890,219.764341,294.774364,283.481648,276.473212,164.943415,266.970832,279.102604
4,356.049563,227.441214,337.982039,281.906495,346.311807,165.883456,371.227807,280.722679,99,424.0,268.503968,219.489491,293.714335,284.456046,275.499248,165.120854,266.477239,277.759174


## 2. Usage of the LabelGenerator class

In [15]:
from app.module import LabelGenerator

In [16]:
lgen = LabelGenerator(
    # This is supposed to be your wireframe data from posenet
    data = feature_data,

    # manually labeled "raw" gesture data with real beginning and real end of movement
    raw_labels = df[["real_start","real_end","label"]],
    
    # associated frame rate of the LabelGenerator. This instance only works with the framerate specified on instantiation
    ms_per_frame = 130    
)

lgen.fit_range(
    # the length of the tolerance range will be 400 ms
    tolerance_range = 400,
    
    # maximum acceptable difference/error in movement length compared to the theoretical movement length (2000 ms)
    # if movement length is smaller than 2000 - max_error or greater than 2000 + max_error, there will be 0-label
    max_error = 400
)

# creates the labeled data set. it can be called with the get_labeled_data method
lgen.set_labels()

# provides 3D labeled data and labels for training. The instance can call X, y, feature_names and final_time
# X --> Array with dimensions [sample size] x [timesteps per sample] x [number of features]
# y --> vector of labels with length [sample size]
# feature_names --> list of the names of the assiciated columns in X
# sequence_end_time --> vector with the number of milliseconds associated with the first dimension of X ([sample size])
lgen.extract_training_data()

In [17]:
# label_df stores information about the labelled time invervals
# if the length deviates too much from 2 seconds, the interval is ignored and not used for training
lgen.label_df

,label,real_start,real_end,from,to,ignore
0,3,18700,20450,20125,20525,False
1,3,20400,22400,22200,22600,False
2,3,22350,24500,24375,24775,False
3,3,26650,28550,28300,28700,False
4,3,30700,32600,32350,32750,False
5,3,34650,36350,36000,36400,False
6,3,38650,40550,40300,40700,False
7,3,42600,44500,44250,44650,False
8,3,46500,48400,48150,48550,False
9,3,50700,52500,52200,52600,False


In [18]:
# label_info contains information about the deviation from the target 2 second length
# upper slack - lower slack is always equal to the tolerance range (here: 400 ms)
lgen.label_info

,diff,l_slack,u_slack,tol_range_indicator
0,-250,-325,75,True
1,0,-200,200,True
2,150,-125,275,True
3,-100,-250,150,True
4,-100,-250,150,True
5,-300,-350,50,True
6,-100,-250,150,True
7,-100,-250,150,True
8,-100,-250,150,True
9,-200,-300,100,True


In [19]:
# labeled data contains all datapoints with the correct labels (or no label at all --> label "0")
lgen.labeled_data.head()

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_last_frame,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y,label
0,362.916626,231.219007,342.953378,283.541312,348.772704,166.245393,372.237121,285.299184,0,0,270.158875,224.355883,294.862550,284.492503,277.058700,166.476502,269.385661,277.767283,0
1,364.180072,231.006023,344.422331,283.465604,347.642222,165.250143,372.912682,285.427752,106,106,270.456436,220.697018,296.207271,283.810974,277.393757,165.318363,268.442911,278.002018,0
2,364.463694,232.132638,344.786367,284.089036,349.140680,165.521388,372.737987,285.656638,110,216,269.989188,222.659280,296.168246,284.376554,276.986625,165.356502,268.550011,278.863787,0
3,361.394807,230.397043,339.481446,282.356428,347.711305,164.712134,373.510148,283.864464,109,325,269.539890,219.764341,294.774364,283.481648,276.473212,164.943415,266.970832,279.102604,0
4,356.049563,227.441214,337.982039,281.906495,346.311807,165.883456,371.227807,280.722679,99,424,268.503968,219.489491,293.714335,284.456046,275.499248,165.120854,266.477239,277.759174,0


In [20]:
# the training data is stored in X (array of 3 dimensions; [sample size] x [timesteps per sample] x [number of features])
# the label-vector is y (1 dim; [sample size])
print(len(lgen.y))
print(lgen.X.shape)
print("")

i = 157
print(lgen.y[i], '\n', lgen.X[i,:,:])

602
(602, 17, 16)

0.0 
 [[326.42521785 218.78132179 301.83115668 270.88615553 307.5536308
  151.77771707 326.00781194 275.3136588  230.30326702 213.41186659
  254.40977385 275.89102748 237.88950791 151.18106611 230.47829949
  263.33233002]
 [324.88313387 218.6399699  300.08482915 271.5614498  305.79546779
  153.53717471 327.82419382 279.63447707 230.50777119 213.9952701
  249.40113503 272.4530815  237.2987216  151.26076627 226.43560111
  264.8068764 ]
 [316.9733346  220.65700056 291.97036865 273.24872648 304.81598305
  152.4398482  324.29400608 279.93485149 231.04483948 209.11208591
  249.04403889 271.89036237 237.56639718 149.26427329 225.22440903
  262.84379844]
 [315.05052561 218.93355129 290.00000809 273.78171312 303.74929413
  152.60073971 321.52488    276.59542966 234.88807739 211.78784367
  249.87311082 276.42599683 238.12417433 148.96540442 226.15210252
  260.65626762]
 [313.61862709 218.64112827 294.57593032 271.36562938 302.61351406
  150.86721811 324.97479669 277.54421248 2

In [21]:
# sequence_end_time stored the last sample timestamp in ms, 
#   when a sequence of 2000ms/130ms (sequence length / framelength) is complete

# first sequence ends with the sample with timestamp 2080 ms
#  then it shifts by 130 ms (the framelength)
lgen.sequence_end_time[0:10]

array([1625, 1727, 1826, 1928, 2034, 2144, 2248, 2359, 2470, 2571])

In [ ]:
lgen.labeled_data.tail(2)

In [ ]:
# the feature names are also stored:
lgen.feature_names